In [103]:
import time
import copy
import heapq

In [104]:
FILE_NAME = "Inputs/input.txt"

In [105]:
f = open(FILE_NAME)

nodes , edges = f.readline().split()
nodes = int(nodes)
edges = int(edges)
type = []

#read edges
edges_list = {}
for i in range(edges):
    u,v = f.readline().split()
    u = int(u)
    v = int(v)
    if u not in edges_list:
        new_list = []
        new_list.append(v)
        edges_list[u] = new_list
    else:
        new_list = []
        new_list.append(v)
        edges_list[u].extend(new_list)
    if v not in edges_list:
        new_list = []
        new_list.append(u)
        edges_list[v] = new_list
    else:
        new_list = []
        new_list.append(u)
        edges_list[v].extend(new_list)


#hard to pass
hard_to_pass = []
hard_to_pass_number = int(f.readline())
hard_to_pass = [int(x) for x in f.readline().split()]

#recipes
follower_number = int(f.readline())
all_recipe = set()
morid = []
recipes = {}
for i in range(follower_number):
    lists = []
    p,q, *arr_follower = [int(x) for x in f.readline().split()]
    morid.append(p)
    all_recipe = all_recipe.union(arr_follower)
    recipes[p] = arr_follower

#start state
start_state = int(f.readline())

all_recipes = list(all_recipe)
f.close()

In [106]:
class State:
    def __init__(self,path,position,pass_recipes,pass_morid,hard_to_pass,hard_time):
        self.path = path
        self.position = position
        self.pass_recipes = pass_recipes
        self.pass_morid = pass_morid
        self.hard_to_pass = hard_to_pass
        self.hard_time = hard_time
    
    def __lt__(self,other):
        return len(self.path) < len(other.path)

    def __eq__(self, state2):
        if self.position == state2.position and self.hard_time == state2.hard_time and self.pass_recipes == state2.pass_recipes and self.pass_morid == state2.pass_morid:
            return True
        else:
            return False

In [107]:
def initialize():
    hard_dict = {}
    pass_recipes = set()
    pass_morid = []
    path = [start_state]
    for hard in hard_to_pass:
        hard_dict[hard] = 0
    start = State(path,start_state,pass_recipes,pass_morid,hard_dict,0)
    return start

In [108]:
def check_hard_state(state):
    if state.position in hard_to_pass:
        if state.hard_to_pass[state.position] > state.hard_time:
            return True
        else:
            state.hard_to_pass[state.position] += 1
    return False

In [109]:
def new_morid(state, neighbour):
    if neighbour in morid:
        if set(recipes[neighbour]).issubset(state.pass_recipes):
            if neighbour not in state.pass_morid:
                state.pass_morid.append(neighbour)
    return state.pass_morid

In [110]:
def new_recipes(state, neighbour):
    if neighbour in all_recipes:
        state.pass_recipes.add(neighbour)
    return state.pass_recipes

In [111]:
def next_states(state):
    next_state = []
    if_in_hard = check_hard_state(state)
    if if_in_hard:
        #current state is hard
        neww = copy.deepcopy(state)
        neww.path.append(neww.position)
        neww.hard_time =neww.hard_time + 1
        next_state.append(neww)
        return next_state
    else:
        for neighbour in edges_list[state.position]:
            #create a new state for each neighbour
            neww = copy.deepcopy(state)
            neww.path.append(neighbour)
            neww.position = neighbour
            neww.pass_recipes = new_recipes(neww, neighbour)
            neww.pass_morid = new_morid(neww, neighbour)
            neww.hard_time = 0
            next_state.append(neww)
        return next_state

In [112]:
def BFS():
    frontier = []
    visited = []
    state = initialize()
    visited_state = 1

    #check if we are in the goal state
    if len(morid) == len(state.pass_morid):
        return len(state.path)-1,visited_state,state.path
    #add the start state to the frontier
    frontier.append(state)

    while frontier:
        #get the first state in the frontier
        state = frontier.pop(0)
        #get all the next states
        next_state = next_states(state)
        for s in next_state:
            checker = False
            if s not in visited:
                for i in range(len(frontier)):
                    if s == frontier[i]:
                        checker = True
                        break
                if checker == False:
                    visited_state += 1
                    #check if we are in the goal state
                    if len(morid) == len(state.pass_morid):
                        return len(state.path)-1,visited_state,state.path

                    #add the state to the frontier
                    frontier.append(s)
        visited.append(state)
    return len(state.path)-1,visited_state,state.path

In [113]:
#check bfs
start = time.time()
cost,visited,bfs_output = BFS()
end = time.time()
start2 = time.time()
cost,visited,bfs_output = BFS()
end2 = time.time()
start3 = time.time()
cost,visited,bfs_output = BFS()
end3 = time.time()
print("file name: ",FILE_NAME)
print("visited states: ",visited)
print("BFS : ", *bfs_output)
print("BFS cost: ", cost)
print("BFS average time: ", (end - start + end2 - start2 + end3 - start3)/3)

file name:  input.txt
visited states:  40
BFS :  1 3 4 5 7 10 11 9 8
BFS cost:  8
BFS average time:  0.0027783711751302085


In [114]:
def DFS(level, state,visited):
    #check if we are in the goal state
    if len(morid) == len(state.pass_morid):
        return True,state.path,visited

    #stop if we reach the max level
    if level < 1:
        return False,None,visited
        
    #get all the next states
    next_state = next_states(state)
    for s in next_state:
        visited += 1
        situation,path,visited = DFS(level - 1, s,visited)
        if situation:
            return True,path,visited
    return False,None,visited


In [115]:
def IDS(level):
    max_level = level
    while max_level >= 0:
        visited = 0
        state = initialize()
        situation,IDS_path,visited_state = DFS(level,state,visited)
        if situation:
            return IDS_path,visited_state,len(IDS_path)-1
        level = level + 1
        max_level = max_level + 1

In [116]:
#check IDS
start = time.time()
IDS_output,visited,cost = IDS(level = 1)
end = time.time()
start2 = time.time()
IDS_output,visited,cost = IDS(level = 1)
end2 = time.time()
start3 = time.time()
IDS_output,visited,cost = IDS(level = 1)
end3 = time.time()
print("file name: ",FILE_NAME)
print("IDS : ", *IDS_output)
print("IDS cost: ", cost)
print("IDS visited: ", visited)
print("IDS average time: ", (end - start + end2 - start2 + end3 - start3)/3)

file name:  input.txt
IDS :  1 3 4 5 7 10 11 9 8
IDS cost:  8
IDS visited:  2519
IDS average time:  0.09938168525695801


In [117]:
def heuristic(state):
    #calculate the heuristic value
    h = []
    for m in morid:
        if m not in state.pass_morid:
            if state.position not in h:
                h.append(state.position)
    for r in all_recipes:
        if r not in state.pass_recipes:
            if state.position not in h:
                h.append(state.position)
    return len(h)

In [118]:
def A_star(alpha):
    frontier = []
    visited = []
    state = initialize()
    visited_state = 1

    #check if we are in the goal state
    if len(morid) == len(state.pass_morid):
        return len(state.path)-1,visited_state,state.path
        
    #add the start state to the frontier
    heapq.heappush(frontier,(alpha*heuristic(state)+len(state.path),state))

    while frontier:
        #get the first state in the frontier
        state = heapq.heappop(frontier)[1]

        #check if we are in the goal state
        if len(morid) == len(state.pass_morid):
            return len(state.path)-1,visited_state,state.path

        #get all the next states
        next_state = next_states(state)
        for s in next_state:
            checker = False
            if s not in visited:
                for i in range(len(frontier)):
                    if s == frontier[i][1]:
                        checker = True
                        break
                if checker == False:
                    visited_state += 1

                    #add the state to the frontier
                    heapq.heappush(frontier,(alpha*heuristic(s)+len(s.path),s))
        visited.append(state)
    return len(state.path)-1,visited_state,state.path

In [119]:
alpha = 7
start = time.time()
cost,visited,A_output = A_star(alpha)
end = time.time()
start2 = time.time()
cost,visited,A_output = A_star(alpha)
end2 = time.time()
start3 = time.time()
cost,visited,A_output = A_star(alpha)
end3 = time.time()
print("file name: ",FILE_NAME)
print("A* : ", *A_output)
print("A* cost: ", cost)
print("A* visited: ", visited)
print("A* average time: ", (end - start + end2 - start2 + end3 - start3)/3)

file name:  input.txt
A* :  1 3 4 5 7 10 11 9 8
A* cost:  8
A* visited:  35
A* average time:  0.00466307004292806
